In [ ]:
import ast, random, spacy
import pandas as pd
from pathlib import Path
from spacy.tokens import DocBin
from spacy.training import Example
from spacy.util import minibatch, compounding
from sklearn.model_selection import train_test_split
# Scorpio

In [ ]:
CSV_PATH = "/content/NER_Dataset.csv"
df = pd.read_csv(CSV_PATH, encoding="latin-1")

df["tokens"] = df["Word"].apply(ast.literal_eval)
df["tags"]   = df["Tag"].apply(ast.literal_eval)

In [ ]:
nlp  = spacy.blank("en")
ner  = nlp.add_pipe("ner")

labels = sorted({t.split("-",1)[1] for row in df.tags for t in row if t != "O"})
for lab in labels:
    ner.add_label(lab)

In [ ]:
def row_to_example(tokens, tags):
    text, offset, spans = "", 0, []
    for tok, tag in zip(tokens, tags):
        if text:
            text += " "
            offset += 1
        start = offset
        text += tok
        end = offset + len(tok)
        if tag != "O":
            spans.append((start, end, tag.split("-",1)[1]))
        offset = end
    doc = nlp.make_doc(text)
    doc.ents = [doc.char_span(s, e, label=l, alignment_mode="contract")
                for s,e,l in spans if doc.char_span(s, e, label=l)]
    return Example.from_dict(doc, {"entities": spans})

examples = [row_to_example(toks, tags)
            for toks, tags in zip(df.tokens, df.tags)]

In [ ]:
train_exs, tmp = train_test_split(examples, test_size=0.2,
                                      random_state=42, shuffle=True)
dev_exs, test_exs  = train_test_split(tmp, test_size=0.5,
                                      random_state=42, shuffle=True)

In [ ]:
for name, exs in [("train", train_exs), ("dev", dev_exs), ("test", test_exs)]:
    db = DocBin(store_user_data=True)
    for ex in exs:
        db.add(ex.reference)
    db.to_disk(f"{name}.spacy")

In [10]:
optimizer = nlp.initialize(get_examples=lambda: train_exs)
N_EPOCHS  = 37

for epoch in range(1, N_EPOCHS + 1):
    random.shuffle(train_exs)
    losses = {}

    for batch in minibatch(train_exs, size=compounding(4.0, 32.0, 1.5)):
        nlp.update(batch, sgd=optimizer, drop=0.2, losses=losses)

    dev_examples = [Example(nlp(ex.reference.text), ex.reference)
                    for ex in dev_exs]
    dev_scores = nlp.evaluate(dev_examples)

    print(
        f"epoch {epoch:02d}  "
        f"loss={losses['ner']:.3f}  "
        f"P={dev_scores['ents_p']:.2f}  "
        f"R={dev_scores['ents_r']:.2f}  "
        f"F1={dev_scores['ents_f']:.2f}"
    )

epoch 01  loss=61284.378  P=0.85  R=0.84  F1=0.85
epoch 02  loss=40865.917  P=0.86  R=0.86  F1=0.86
epoch 03  loss=36770.088  P=0.87  R=0.85  F1=0.86
epoch 04  loss=34194.790  P=0.87  R=0.86  F1=0.87
epoch 05  loss=32499.618  P=0.87  R=0.86  F1=0.86
epoch 06  loss=30931.744  P=0.87  R=0.85  F1=0.86
epoch 07  loss=29772.791  P=0.88  R=0.86  F1=0.87
epoch 08  loss=28540.543  P=0.88  R=0.86  F1=0.87
epoch 09  loss=27717.044  P=0.87  R=0.87  F1=0.87
epoch 10  loss=26981.188  P=0.87  R=0.88  F1=0.87
epoch 11  loss=26049.764  P=0.88  R=0.87  F1=0.87
epoch 12  loss=25342.150  P=0.88  R=0.87  F1=0.88
epoch 13  loss=25053.035  P=0.88  R=0.87  F1=0.88
epoch 14  loss=24272.777  P=0.88  R=0.87  F1=0.87
epoch 15  loss=24019.191  P=0.88  R=0.87  F1=0.88
epoch 16  loss=23758.517  P=0.88  R=0.86  F1=0.87
epoch 17  loss=22932.794  P=0.88  R=0.87  F1=0.87
epoch 18  loss=22706.040  P=0.87  R=0.87  F1=0.87
epoch 19  loss=22285.696  P=0.88  R=0.87  F1=0.87
epoch 20  loss=22076.073  P=0.88  R=0.87  F1=0.87


In [11]:
test_score = nlp.evaluate([Example(nlp(ex.reference.text), ex.reference)
                           for ex in test_exs])
print("TEST  F1={ents_f:.2f}  P={ents_p:.2f}  R={ents_r:.2f}".format(**test_score))

TEST  F1=0.87  P=0.87  R=0.88


In [12]:
out_dir = Path("best_ner_model")
nlp.to_disk(out_dir)
print(f"model saved to {out_dir.resolve()}")

model saved to /content/best_ner_model


In [22]:
# Creates a small, human-editable template
!python -m spacy init config base_config.cfg \
       --lang en \
       --pipeline ner \
       --optimize efficiency


⚠ To generate a more effective transformer-based config (GPU-only),
install the spacy-transformers package and re-run this command. The config
generated now does not use transformers.
ℹ Generated config template specific for your use case
- Language: en
- Pipeline: ner
- Optimize for: efficiency
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
base_config.cfg
You can now add your data and train your pipeline:
python -m spacy train base_config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [23]:
!python -m spacy init fill-config base_config.cfg config.cfg

⚠ Nothing to auto-fill: base config is already complete
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [26]:
!python -m spacy train config.cfg \
     --output ./output \
     --paths.train ./train.spacy \
     --paths.dev   ./dev.spacy

ℹ Saving to output directory: output
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     51.41    1.68    3.56    1.10    0.02
  0     200         96.42   3189.72   68.91   71.05   66.89    0.69
  0     400        281.63   2072.62   77.36   78.31   76.45    0.77
  0     600        207.21   2145.63   79.06   79.30   78.81    0.79
  0     800        247.47   2567.05   81.46   83.33   79.67    0.81
  0    1000        292.26   2779.74   82.21   84.39   80.15    0.82
  0    1200        332.93   3181.09   83.18   84.26   82.14    0.83
  0    1400        396.53   3680.62   82.92   83.97   81.91    0.83
  0    1600        474.97   438

In [27]:
!python -m spacy evaluate output/model-best ./test.spacy --output metrics.json

ℹ Using CPU

================================== Results ==================================

TOK     100.00
NER P   88.25 
NER R   86.87 
NER F   87.55 
SPEED   20269 


=============================== NER (per type) ===============================

          P       R       F
gpe   96.10   93.59   94.83
per   86.59   91.60   89.02
geo   86.90   91.92   89.34
tim   94.58   89.71   92.08
org   84.78   74.98   79.58
eve   48.94   39.66   43.81
art    5.88    1.03    1.75
nat   60.00   33.33   42.86

✔ Saved results to metrics.json
